Scenario: The MSCI ESG Score Aggregator

You need to generate a report for a list of 5 Companies (e.g., Apple, Microsoft, etc.). For each company, you must:

Fetch Fundamental Data: Get the Market Cap and Sector from the internal MSCI database (takes ~0.8s, Blocking).

Fetch Carbon Emission Data: Get data from an external API (takes ~1.2s, Async).

Fetch Governance Ratings: Get ratings from a third-party partner (takes ~1.0s, Async).

Calculate Final Score: A CPU-heavy calculation that combines these values.

Update Central Cache: Store the result in a shared dictionary (index_cache).

Constraints:

Concurrency: You can only make 2 external API calls at a time (to respect rate limits).

Timeout: If a company's data doesn't arrive within 2.0 seconds, skip that company and log a "Data Incomplete" error.

Integrity: The index_cache must be protected from race conditions.

In [2]:
import time

index_cache = {}

def fetch_dundamentals(company):
    # DB call below to get market cap & sector
    time.sleep(0.8)
    return {"ticker": company, "mkt_cap": "1.2T", "sector": "Tech"}

def fetch_emission_data(company):
    # API call to get carbon emission data
    time.sleep(1.2)
    return {"carbon_footprint": "Low"}

def fetch_governance_ratings(company):
    # API call to get carbon emission data
    time.sleep(1)
    return {"gov_score": "AA"}

def calculate_final_score(company):
    # CPU-heavy calculation that combines values in data
    fundamental_data = fetch_dundamentals(company)
    emission_data = fetch_emission_data(company)
    governance_data = fetch_governance_ratings(company)
    final_data = f"Final data - {fundamental_data}, {emission_data}, {governance_data}"
    return final_data

def main():
    companies = ['Apple', 'Microsoft', 'Meta', 'Amazon', 'Tesla']
    start_time = time.perf_counter()
    for company in companies:
        index_cache[company] = calculate_final_score(company)
    print(index_cache)
    end_time = time.perf_counter()
    print(f"\nCache size: {len(index_cache)}")
    print(f"Total time: {end_time - start_time:.2f}s")
    
if __name__ == "__main__":
    main()



{'Apple': "Final data - {'ticker': 'Apple', 'mkt_cap': '1.2T', 'sector': 'Tech'}, {'carbon_footprint': 'Low'}, {'gov_score': 'AA'}", 'Microsoft': "Final data - {'ticker': 'Microsoft', 'mkt_cap': '1.2T', 'sector': 'Tech'}, {'carbon_footprint': 'Low'}, {'gov_score': 'AA'}", 'Meta': "Final data - {'ticker': 'Meta', 'mkt_cap': '1.2T', 'sector': 'Tech'}, {'carbon_footprint': 'Low'}, {'gov_score': 'AA'}", 'Amazon': "Final data - {'ticker': 'Amazon', 'mkt_cap': '1.2T', 'sector': 'Tech'}, {'carbon_footprint': 'Low'}, {'gov_score': 'AA'}", 'Tesla': "Final data - {'ticker': 'Tesla', 'mkt_cap': '1.2T', 'sector': 'Tech'}, {'carbon_footprint': 'Low'}, {'gov_score': 'AA'}"}

Cache size: 5
Total time: 15.04s


Asynchronnous way

In [ ]:
import time
import asyncio

index_cache = {}

index_cache_lock = asyncio.Lock()
thread_limit = asyncio.Semaphore(2)

def fetch_dundamentals(company):
    # DB call below to get market cap & sector
    time.sleep(0.8)
    return {"ticker": company, "mkt_cap": "1.2T", "sector": "Tech"}

async def fetch_emission_data(company):
    # API call to get carbon emission data
    await asyncio.sleep(1.2)
    return {"carbon_footprint": "Low"}

async def fetch_governance_ratings(company):
    # API call to get governance data
    await asyncio.sleep(1)
    return {"gov_score": "AA"}

async def calculate_final_score(company):
    # db call no limit, api call a limit of 2
    try:
        async def restricted_fetch():
            result_db = asyncio.to_thread(fetch_dundamentals, company)
            

            async def api_calls():
                async with thread_limit:
                    return await asyncio.gather(
                        fetch_emission_data(company), 
                        fetch_governance_ratings(company)
                        )
            
            return await asyncio.gather(result_db, api_calls())
    
        raw_data = await asyncio.wait_for(restricted_fetch(), timeout=2)
        fundamental_data = raw_data[0]
        emission_data, governance_data = raw_data[1]
            
        final_data = f"Score for {company}: {fundamental_data}, {emission_data}, {governance_data}"
        async with index_cache_lock:
                index_cache[company] = final_data

    except asyncio.TimeoutError:
        async with index_cache_lock:
            index_cache[company] = "Error: Timeout"
    except Exception as e:
        async with index_cache_lock:
            index_cache[company] = f"Error: {str(e)}"


async def main():
    companies = ['Apple', 'Microsoft', 'Meta', 'Amazon', 'Tesla']
    start_time = time.perf_counter()

    k = [calculate_final_score(company) for company in companies]
    await asyncio.gather(*k)

    print(index_cache)
    end_time = time.perf_counter()
    print(f"\nCache size: {len(index_cache)}")
    print(f"Total time: {end_time - start_time:.2f}s")
    
if __name__ == "__main__":
    await main()        # Use asyncio.run(main())



{'Apple': "Score for Apple: {'ticker': 'Apple', 'mkt_cap': '1.2T', 'sector': 'Tech'}, {'carbon_footprint': 'Low'}, {'gov_score': 'AA'}", 'Microsoft': "Score for Microsoft: {'ticker': 'Microsoft', 'mkt_cap': '1.2T', 'sector': 'Tech'}, {'carbon_footprint': 'Low'}, {'gov_score': 'AA'}", 'Tesla': 'Error: Timeout', 'Meta': 'Error: Timeout', 'Amazon': 'Error: Timeout'}

Cache size: 5
Total time: 2.00s
